In [1]:
from subgrounds import Subgrounds
from subgrounds.pagination import ShallowStrategy
import pandas as pd


In [2]:
endpoint = 'https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-v2'
sg = Subgrounds()
endpoint = endpoint
subgraph = sg.load_subgraph(endpoint)

CONFIG_COMMON_SWAPS_TOKENS = {
    "ethereum": {
        # [GHO]
        'search_tokens': ["0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f", ],
    }
}


In [8]:
# Build the query
chain = 'ethereum'
config = CONFIG_COMMON_SWAPS_TOKENS[chain]
last_materialized_timestamp = 1693362587
last_materialized_block = 18024449


tokens_where = []
tokens_where.append({
    'tokenIn_in': config['search_tokens'],
    'timestamp_gt': last_materialized_timestamp
})
tokens_where.append({
    'tokenOut_in': config['search_tokens'],
    'timestamp_gt': last_materialized_timestamp
})

swaps_data = pd.DataFrame()
for w in tokens_where:
    swaps = subgraph.Query.swaps(
        where=w,
        first=1000000
    )

    field_paths = [
        swaps.id,
        swaps.tx,
        swaps.timestamp,
        swaps.poolId.id,
        swaps.poolId.address,
        swaps.poolId.swapFee,
        swaps.caller,
        swaps.userAddress.id,
        swaps.tokenIn,
        swaps.tokenOut,
        swaps.tokenInSym,
        swaps.tokenOutSym,
        swaps.tokenAmountIn,
        swaps.tokenAmountOut,
        swaps.valueUSD
    ]

    part = sg.query_df(field_paths, pagination_strategy=ShallowStrategy)
    swaps_data = pd.concat([swaps_data, part])

# swaps_data
swaps_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 341 entries, 0 to 183
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   swaps_id              341 non-null    object 
 1   swaps_tx              341 non-null    object 
 2   swaps_timestamp       341 non-null    int64  
 3   swaps_poolId_id       341 non-null    object 
 4   swaps_poolId_address  341 non-null    object 
 5   swaps_poolId_swapFee  341 non-null    float64
 6   swaps_caller          341 non-null    object 
 7   swaps_userAddress_id  341 non-null    object 
 8   swaps_tokenIn         341 non-null    object 
 9   swaps_tokenOut        341 non-null    object 
 10  swaps_tokenInSym      341 non-null    object 
 11  swaps_tokenOutSym     341 non-null    object 
 12  swaps_tokenAmountIn   341 non-null    float64
 13  swaps_tokenAmountOut  341 non-null    float64
 14  swaps_valueUSD        341 non-null    float64
dtypes: float64(4), int64(1)

In [13]:
# cp = swaps_data.copy()


In [18]:
swaps_data = cp.copy()
swaps_data.rename(columns={
    'swaps_tx': 'tx_hash',
    'swaps_logIndex': 'event_index',
    'swaps_timestamp': 'block_unixtimestamp',
    'swaps_transaction_blockNumber': 'block_number',
    'swaps_poolId_address': 'pool_address',
    'swaps_poolId_id': 'pool_id',
    'swaps_poolId_swapFee': 'fee_tier',
    'swaps_caller': 'caller_address',
    'swaps_userAddress_id': 'sender_address',
    'swaps_tokenIn': 'token_in_address',
    'swaps_tokenOut': 'token_out_address',
    'swaps_tokenInSym': 'token_in_symbol',
    'swaps_tokenOutSym': 'token_out_symbol',
    'swaps_tokenAmountIn': 'token_in_amount',
    'swaps_tokenAmountOut': 'token_out_amount',
}, inplace=True)

swaps_data['block_timestamp'] = pd.to_datetime(
    swaps_data['block_unixtimestamp'], unit='s')
swaps_data['chain'] = chain
swaps_data['platform'] = 'uniswap_v3'
swaps_data['is_aggregator'] = False
swaps_data['recipient_address'] = swaps_data.sender_address
# swaps_data['event_index'] = swaps_data.event_index.astype(int)
swaps_data['event_index'] = swaps_data.apply(
    lambda row: row['swaps_id'].replace(row['tx_hash'], ''), axis=1)


swaps_data['token_in_amount_usd'] = swaps_data.swaps_valueUSD
swaps_data['token_out_amount_usd'] = swaps_data.swaps_valueUSD
swaps_data['token_in_price_usd'] = swaps_data.swaps_valueUSD / \
    swaps_data.token_in_amount
swaps_data['token_out_price_usd'] = swaps_data.swaps_valueUSD / \
    swaps_data.token_out_amount
swaps_data['block_number'] = None

swaps_data = swaps_data[[
    "tx_hash",
    "event_index",
    "block_timestamp",
    "block_unixtimestamp",
    "block_number",
    "chain",
    "platform",
    "pool_address",
    "is_aggregator",
    "caller_address",
    "recipient_address",
    "token_in_address",
    "token_out_address",
    "token_in_symbol",
    "token_out_symbol",
    "token_in_amount",
    "token_in_amount_usd",
    "token_in_price_usd",
    "token_out_amount",
    "token_out_amount_usd",
    "token_out_price_usd"
]]

# swaps_data.sort_values(by=['block_number', 'event_index'], inplace=True)
# last_block = swaps_data.block_number.max()
# last_timestamp = swaps_data.block_unixtimestamp.max()


In [19]:
swaps_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 341 entries, 0 to 183
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   tx_hash               341 non-null    object        
 1   event_index           341 non-null    object        
 2   block_timestamp       341 non-null    datetime64[ns]
 3   block_unixtimestamp   341 non-null    int64         
 4   block_number          0 non-null      object        
 5   chain                 341 non-null    object        
 6   platform              341 non-null    object        
 7   pool_address          341 non-null    object        
 8   is_aggregator         341 non-null    bool          
 9   caller_address        341 non-null    object        
 10  recipient_address     341 non-null    object        
 11  token_in_address      341 non-null    object        
 12  token_out_address     341 non-null    object        
 13  token_in_symbol     

In [20]:
swaps_data


,tx_hash,event_index,block_timestamp,block_unixtimestamp,block_number,chain,platform,pool_address,is_aggregator,caller_address,...,token_in_address,token_out_address,token_in_symbol,token_out_symbol,token_in_amount,token_in_amount_usd,token_in_price_usd,token_out_amount,token_out_amount_usd,token_out_price_usd
0,0x02376edad7074e9c866f204c143d4ad7797eb8c0cecb...,298,2023-09-06 03:30:35,1693971035,None,ethereum,uniswap_v3,0xbe19d87ea6cd5b05bbc34b564291c371dae96747,False,0x4ca37dfef52d3ff6dcaed147e05c0903a4fa4488,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x79c58f70905f734641735bc61e45c19dd9ad60bc,GHO,USDC-DAI-USDT,7000.000000,6781.839936,0.968834,6763.654047,6781.839936,1.002689
1,0x0248686dfa9d64576ef035e94ce0cf4ccede28fed0f6...,260,2023-09-08 18:03:47,1694196227,None,ethereum,uniswap_v3,0xbe19d87ea6cd5b05bbc34b564291c371dae96747,False,0x028b2b9adca1bd1263086b0b221713a684968817,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x79c58f70905f734641735bc61e45c19dd9ad60bc,GHO,USDC-DAI-USDT,36108.000000,35113.134662,0.972448,35051.500205,35113.134662,1.001758
2,0x032d9eb1c38ca2728d63f86d06287444805635ac903e...,52,2023-09-03 10:00:11,1693735211,None,ethereum,uniswap_v3,0x7d98f308db99fdd04bbf4217a4be8809f38faa64,False,0x7421ee752e6da4f74ce5fa7f2887a4c995f13613,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,GHO,wstETH,2374.217779,2264.932047,0.953970,1.224791,2264.932047,1849.239757
3,0x035e80a931ced411b7bfe1604817a800cf7dee740e84...,431,2023-09-02 11:32:11,1693654331,None,ethereum,uniswap_v3,0x3fa8c89704e5d07565444009e5d9e624b40be813,False,0xa06c3c08a19e51b33309eddfb356c33ead8517a3,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,GHO,LUSD,10045.516374,9851.560366,0.980692,9839.764612,9851.560366,1.001199
4,0x03a40ae9d6e8a1b367ef8f234998440bafd580ef7e84...,109,2023-09-07 10:26:47,1694082407,None,ethereum,uniswap_v3,0x8353157092ed8be69a9df8f95af097bbf33cb2af,False,0x6512055c7e562e471f58932b8d575d19fbdbb6a0,...,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0xdac17f958d2ee523a2206206994597c13d831ec7,GHO,USDT,3000.000000,2915.960504,0.971987,2915.960504,2915.960504,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,0xf7d03b2cd2c18d5b67104156bb09f1eb89fcd6e0dc84...,161,2023-09-03 09:32:35,1693733555,None,ethereum,uniswap_v3,0xbe19d87ea6cd5b05bbc34b564291c371dae96747,False,0x8af700ba841f30e0a3fcb0ee4c4a9d223e1efa05,...,0x79c58f70905f734641735bc61e45c19dd9ad60bc,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,USDC-DAI-USDT,GHO,21896.679611,21933.654653,1.001689,22681.416463,21933.654653,0.967032
180,0xf954ab990c06a4eced3201886ca70f5fba2e36921a15...,0,2023-09-08 21:31:59,1694208719,None,ethereum,uniswap_v3,0x8353157092ed8be69a9df8f95af097bbf33cb2af,False,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,USDC,GHO,77670.674266,77670.674266,1.000000,79586.486568,77670.674266,0.975928
181,0xfb23def3dcfdce4db93739c2b71fead2207931ec484b...,18,2023-09-04 17:12:35,1693847555,None,ethereum,uniswap_v3,0xbe19d87ea6cd5b05bbc34b564291c371dae96747,False,0xc1febdfb461529b086a9adc3f95912acd87f14bd,...,0x79c58f70905f734641735bc61e45c19dd9ad60bc,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,USDC-DAI-USDT,GHO,26850.465464,27062.091625,1.007882,27642.752878,27062.091625,0.978994
182,0xfd8baa4ac72026c68d0a18f50050471790d25611b6ac...,94,2023-09-14 04:53:35,1694667215,None,ethereum,uniswap_v3,0x8353157092ed8be69a9df8f95af097bbf33cb2af,False,0x4339889fd9dfca20a423fba011e9dff1c856caeb,...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,USDC,GHO,3906.526207,3906.526207,1.000000,4039.193774,3906.526207,0.967155


In [61]:
l = int(swaps_data.block_number.max())
